In [112]:
import rioxarray
import rasterio.enums
import geopandas
import shapely
import pathlib
import numpy

In [114]:
import sys
sys.path;

In [310]:
input_path=pathlib.Path(r"C:\Local\data\DEMs\flood_defence_and_patching")
crs=2193

In [270]:
def _write_netcdf_conventions_in_place(dem, crs):

    dem.rio.write_crs(crs, inplace=True)
    dem.rio.write_transform(inplace=True)
    if "z" in dem:
        dem.z.rio.write_crs(crs, inplace=True)
        dem.z.rio.write_nodata(numpy.nan, encoded=True, inplace=True)
    if "data_source" in dem:
        dem.data_source.rio.write_crs(crs, inplace=True)
        dem.data_source.rio.write_nodata(numpy.nan, encoded=True, inplace=True)
    if "lidar_source" in dem:
        dem.lidar_source.rio.write_crs(crs, inplace=True)
        dem.lidar_source.rio.write_nodata(numpy.nan, encoded=True, inplace=True)

def save_dataset(filename: pathlib.Path, dem):
        assert not any(
            array.rio.crs is None for array in dem.data_vars.values()
        ), "all DataArray variables of a xarray.Dataset must have a CRS"

        _write_netcdf_conventions_in_place(dem, dem.rio.crs)
        if filename.suffix.lower() == ".nc":
            dem.to_netcdf(filename, format="NETCDF4", engine="netcdf4")
        elif filename.suffix.lower() == ".tif":
            for key, array in dem.data_vars.items():
                filename_layer = (
                    filename.parent / f"{filename.stem}_{key}{filename.suffix}"
                )
                array.encoding = {
                    "dtype": array.dtype,
                    "grid_mapping": array.encoding["grid_mapping"],
                    "rasterio_dtype": array.dtype,
                }
                array.rio.to_raster(filename_layer)

def save_dataarray(filename: pathlib.Path, dem):
    assert dem.rio.crs is not None, "dem must have a CRS"
    
    dem.rio.write_transform(inplace=True)
    dem.rio.write_nodata(dem.rio.nodata, encoded=True, inplace=True)
    
    if filename.suffix.lower() == ".nc":
        dem.to_netcdf(filename, format="NETCDF4", engine="netcdf4")
    elif filename.suffix.lower() == ".tif":
        dem.encoding = {
            "dtype": dem.dtype,
            "grid_mapping": dem.encoding["grid_mapping"],
            "rasterio_dtype": dem.dtype,
        }
        dem.rio.to_raster(filename)

## Save out patches

In [ ]:
output_path = pathlib.Path(r"C:\Local\repos\GeoFabrics\tests\test_add_patches\data")

In [122]:
# clip origional
origional = rioxarray.rioxarray.open_rasterio(input_path / "merge" / "8m_geofabric.nc").squeeze("band", drop=True)
x1 = 1919400; y1 = 5610000
x2 = 1920703; y2 = 5611400
roi = geopandas.GeoDataFrame(geometry=[shapely.geometry.Polygon([[x1,y1],[x1,y2],[x2,y2],[x2,y1]])], crs=crs)
origional = origional.rio.clip(roi.geometry)
save_dataset(dem=origional, filename=output_path / "initial_dem.nc")

In [314]:
# clip neg
neg = rioxarray.rioxarray.open_rasterio(input_path / "Ngaruroro_neg50cm.nc", mask=True).squeeze("band", drop=True)
x1 = 1920000; y1 = 5610000
x2 = 1920703; y2 = 5611400
roi = geopandas.GeoDataFrame(geometry=[shapely.geometry.Polygon([[x1,y1],[x1,y2],[x2,y2],[x2,y1]])], crs=crs)
neg=neg.rio.clip(roi.geometry)  # .sel(x=origional.x, y=origional.y, method="nearest")
neg=neg.where(neg!=neg.rio.nodata, numpy.nan)
neg.rio.write_crs(crs, inplace=True);
neg.rio.write_nodata(numpy.nan, inplace=True, encoded=True);
save_dataarray(dem=neg, filename=output_path / "patch_1.nc")

In [328]:
# clip and upsample pos
pos = rioxarray.rioxarray.open_rasterio(input_path / "Ngaruroro_pos50cm.nc", mask=True).squeeze("band", drop=True)
x1 = 1919400; y1 = 5610000
x2 = 1920100; y2 = 5611400
roi = geopandas.GeoDataFrame(geometry=[shapely.geometry.Polygon([[x1,y1],[x1,y2],[x2,y2],[x2,y1]])], crs=2193)
pos = pos.rio.clip(roi.geometry)
pos = pos.where(pos!=pos.rio.nodata, numpy.nan)
pos.rio.write_crs(crs, inplace=True);
upscale_factor = 2
pos_upsampled = pos.rio.reproject(pos.rio.crs, shape=(pos.rio.height*upscale_factor, pos.rio.width*upscale_factor), resampling=rasterio.enums.Resampling.bilinear)
save_dataarray(dem=pos_upsampled, filename=output_path / "patch_2.tif")
pos_upsampled.rio.resolution()

(4.0, -4.0)

## Save out Stopbanks

In [444]:
output_path = pathlib.Path(r"C:\Local\repos\GeoFabrics\tests\test_add_stopbanks_waikanae\data")

In [436]:
stopbanks = geopandas.read_file(input_path / "Hawke_s_Bay_Stopbanks.geojson").to_crs(2193)
x1 = 1934000; y1 = 5611500
x2 = 1932800; y2 = 5609600
roi = geopandas.GeoDataFrame(geometry=[shapely.geometry.Polygon([[x1,y1],[x1,y2],[x2,y2],[x2,y1]])], crs=crs)
stopbanks = stopbanks.clip(roi)
stopbanks[["geometry"]].to_file(filename=output_path / "stopbanks.geojson")

In [446]:
stopbanks = geopandas.read_file(input_path / "stopbanks" / "NZIS_v1_new.gpkg").to_crs(2193)
x1 = 1771850; y1 = 5472250
x2 = 1772150; y2 = 5472600
roi = geopandas.GeoDataFrame(geometry=[shapely.geometry.Polygon([[x1,y1],[x1,y2],[x2,y2],[x2,y1]])], crs=crs)
stopbanks = stopbanks.clip(roi)
stopbanks[["geometry"]].to_file(filename=output_path / "stopbanks.geojson")

In [442]:
stopbanks.head(2)

,NZIS_Source,NZIS_Region,geometry
3038,Council,Wellington,"MULTILINESTRING ((1771875.612 5472485.270, 177..."
3045,Council,Wellington,"LINESTRING (1772150.000 5472475.014, 1772148.1..."


In [448]:
stopbanks.explore()